In [1]:
import yt
import numpy as np
from yt.units import K, mp, kb, mh, planck_constant, cm, m, s, g, erg
from quokka2s import *
import numpy as np
import yt

from quokka2s.pipeline.prep import config as cfg
from quokka2s.pipeline.prep import physics_fields as phys

ds = yt.load("~/quokka_postprocessing/examples/halpha_analysis/plt263168")
ds.derived_field_list

yt : [INFO     ] 2025-12-05 21:45:23,116 Parameters: current_time              = 3180193002532269.5
yt : [INFO     ] 2025-12-05 21:45:23,116 Parameters: domain_dimensions         = [128 128 512]
yt : [INFO     ] 2025-12-05 21:45:23,116 Parameters: domain_left_edge          = [ 0.000e+00  0.000e+00 -6.036e+21]
yt : [INFO     ] 2025-12-05 21:45:23,152 Parameters: domain_right_edge         = [3.018e+21 3.018e+21 6.036e+21]


[('StochasticStellarPop_particles', 'birth_time'),
 ('StochasticStellarPop_particles', 'death_time'),
 ('StochasticStellarPop_particles', 'luminosity'),
 ('StochasticStellarPop_particles', 'mass'),
 ('StochasticStellarPop_particles', 'mesh_id'),
 ('StochasticStellarPop_particles', 'particle_cpu'),
 ('StochasticStellarPop_particles', 'particle_id'),
 ('StochasticStellarPop_particles', 'particle_index'),
 ('StochasticStellarPop_particles', 'particle_int_comp0'),
 ('StochasticStellarPop_particles', 'particle_ones'),
 ('StochasticStellarPop_particles', 'particle_position'),
 ('StochasticStellarPop_particles', 'particle_position_cylindrical_radius'),
 ('StochasticStellarPop_particles', 'particle_position_cylindrical_theta'),
 ('StochasticStellarPop_particles', 'particle_position_cylindrical_z'),
 ('StochasticStellarPop_particles', 'particle_position_relative_x'),
 ('StochasticStellarPop_particles', 'particle_position_relative_y'),
 ('StochasticStellarPop_particles', 'particle_position_relat

In [2]:
ds = yt.load(cfg.YT_DATASET_PATH)
ad = ds.all_data()
density = ad[('gas', 'density')]
E_int_density = ad[('gas', 'internal_energy_density')]
E_int_density
# yt.SlicePlot(ds, 'z', ('boxlib', 'gasDensity'))


yt : [INFO     ] 2025-12-05 21:45:23,858 Parameters: current_time              = 3180193002532269.5
yt : [INFO     ] 2025-12-05 21:45:23,858 Parameters: domain_dimensions         = [128 128 512]
yt : [INFO     ] 2025-12-05 21:45:23,859 Parameters: domain_left_edge          = [ 0.000e+00  0.000e+00 -6.036e+21]
yt : [INFO     ] 2025-12-05 21:45:23,859 Parameters: domain_right_edge         = [3.018e+21 3.018e+21 6.036e+21]


unyt_array([8.18744811e-14, 7.86973558e-14, 8.11166920e-14, ...,
       2.87125197e-14, 2.74111803e-14, 2.76248397e-14], shape=(8388608,), units='dyn/cm**2')

In [9]:
import yt
import os
from yt.units import K, mh, kb
from quokka2s.pipeline.prep import config as cfg
from quokka2s.pipeline.prep import physics_fields
from quokka2s.tables import load_table
from quokka2s.tables.lookup import TableLookup

os.chdir("/Users/baochen/quokka_postprocessing")
ds = yt.load(cfg.YT_DATASET_PATH)
physics_fields.add_all_fields(ds)

ad = ds.all_data()

# 质量密度 rho，单位 g/cm^3
rho = ad[('gas', 'density')].to('g/cm**3').value          # unyt_array

# 氢核数密度 nH，若没有现成字段则用 X_H=0.71 派生：
X_H = 0.74

nH = (rho * X_H / mh).in_cgs().value                 # unyt_array

# 内能密度 E_int，确认单位是 erg/cm^3
E_int_density = ad[('gas', 'internal_energy_density')].to('erg/cm**3').value

# nH_sample = nH[:100].to_value()           # cm^-3
# rho_sample = rho[:100].to_value()         # g/cm^3
# Eint_sample = E_int_density[:100].to_value()  # erg/cm^3

colDen = ad[('gas', 'column_density_H')].in_cgs().value # 若无现成字段，需要自己积分或用已有列密度

tbl = load_table(cfg.DESPOTIC_TABLE_PATH)
lookup = TableLookup(tbl)
T_min, T_max = tbl.T_values.min(), tbl.T_values.max()

gamma = 5/3

mh = mh.in_cgs().value
kb = kb.in_cgs().value

def iterate_T_mu(nH_arr, col_arr, rho_arr, Eint_arr, T_init=1e3, max_iter=10, tol=1e-3):
    T = np.full_like(nH_arr, T_init, dtype=float)
    mu = np.full_like(nH_arr, np.nan, dtype=float)
    for _ in range(max_iter):
        T = np.clip(T, T_min, T_max)
        mu = lookup.mu(nH_arr, col_arr, T)
        T_new = (gamma - 1.0) * (Eint_arr / rho_arr) * mu * (mh / kb)
        converged = np.abs(T_new - T) / np.maximum(T, 1e-10) < tol
        T[~converged] = T_new[~converged]
        if converged.all():
            break
    return T, mu




yt : [INFO     ] 2025-12-06 04:14:45,369 Parameters: current_time              = 3180193002532269.5
yt : [INFO     ] 2025-12-06 04:14:45,369 Parameters: domain_dimensions         = [128 128 512]
yt : [INFO     ] 2025-12-06 04:14:45,370 Parameters: domain_left_edge          = [ 0.000e+00  0.000e+00 -6.036e+21]
yt : [INFO     ] 2025-12-06 04:14:45,370 Parameters: domain_right_edge         = [3.018e+21 3.018e+21 6.036e+21]


AxisError: axis 1 is out of bounds for array of dimension 1

In [ ]:
from pathlib import Path
print(cfg.DESPOTIC_TABLE_PATH)
print(Path(cfg.DESPOTIC_TABLE_PATH).resolve())


output_tables_NL99GC_iterate_25/despotic_table.npz
/Users/baochen/quokka_postprocessing/quokka2s/src/quokka2s/tables/output_tables_NL99GC_iterate_25/despotic_table.npz


In [5]:
ad = ds.all_data()
dx = ad[('boxlib', 'dx')]
print(dx)
print(dx.shape)


[2.3578125e+19 2.3578125e+19 2.3578125e+19 ... 2.3578125e+19 2.3578125e+19
 2.3578125e+19] cm
(8388608,)


In [16]:
from yt.units import c
print(c)

c


In [4]:

print("has mu?", hasattr(ds, "mu"), ds.mu if hasattr(ds, "mu") else None)
print("default_species_fields =", ds.default_species_fields)
from yt.utilities.chemical_formulas import compute_mu
print("mu used =", getattr(ds, "mu", compute_mu(ds.default_species_fields)))


has mu? False None
default_species_fields = None
mu used = 0.5924489101195808


In [ ]:

numberDen = ds[('gas', 'number_density')]


density =  ds[('gas', 'density')]

print(numberDen)

yt : [INFO     ] 2025-10-26 17:10:49,671 Parameters: current_time              = 3180193002532269.5
yt : [INFO     ] 2025-10-26 17:10:49,672 Parameters: domain_dimensions         = [128 128 512]
yt : [INFO     ] 2025-10-26 17:10:49,673 Parameters: domain_left_edge          = [ 0.000e+00  0.000e+00 -6.036e+21]
yt : [INFO     ] 2025-10-26 17:10:49,675 Parameters: domain_right_edge         = [3.018e+21 3.018e+21 6.036e+21]


KeyError: ('gas', 'number_density')